# 1. Importación de paquetes

In [40]:
#Import packages
from modules import preprocess
from modules import training

# 2. Preprocesamiento del *dataset*: [Room Occupancy Estimation Data Set](https://www.kaggle.com/ananthr1/room-occupancy-estimation-data-set)

## Paso 1: Lectura del *dataset* y obtención de datos

In [41]:
df = preprocess.load_csv_data('Occupancy_Estimation.csv')
df.shape

(10129, 19)

In [42]:
df.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 2: Recodificación de la columna *RoomOccupancyCount*

En este caso, la columna que contiene la variable de salida (*RoomOccupancyCount*) cuenta el número de personas en la sala. Para simplificar, nuestro objetivo va a ser detectar si la sala está vacía o, en cambio, hay alguien (no el número exacto de personas).

Por lo tanto, se **recodificará cualquier valor mayor que 0 en esa columna como 1**, de tal forma que la variable de salida sea binaria:
- 0 -> ausencia.
- 1 -> presencia.

In [43]:
# Show "Room_Occupancy_Count" before processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

In [44]:
#Process "Room_Occupancy_Count"
preprocess.recode_dataset_output(df)

# Show "Room_Occupancy_Count" after processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int32

## Paso 3: Eliminación de las columnas temporales *Date* y *Time*

Respecto a los datos ofrecidos por las columas *Date* y *Time*, vamos a filtrarlos y dejarlos fuera del proceso. Ya que, aunque los incluyeramos (por ejemplo, juntando ambas partes en una sola columna y convirtiéndolo a formato 'epoch', como long int), los valores de muestras sucesivas de esa columna estarían totalmente correlados entre sí y, como consecuencia, fastidiaríamos a la mayoría de algoritmos que vamos a emplear posteriormente.

Se debe tener en cuenta que no nos estamos olvidando de la información temporal para considerar como si cada valor muestreado (fila de la tabla) fuese independiente de las demás filas. Sabemos que eso no es así, pero estamos considerando que esa información temporal no nos ofrece valor añadido para predecir si la habitación está ocupada o vacía. En caso contrario, se tendrían que usar modelos bastante más complicados para considerar esa relación temporal que indica que, en realidad, las muestras de dos filas adyacentes son consecutivas en el tiempo.

In [45]:
# Filter Date and Time columns
preprocess.remove_time_columns(df)

# Show the result
df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 4: División del *dataset* en *train* y *test*

In [46]:
# Split dataset into train and test
train_df, test_df = preprocess.split_dataset(df)
train_df.shape, test_df.shape

((7596, 17), (2533, 17))

In [47]:
train_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
5671,25.25,25.25,25.06,25.88,15,18,70,49,0.07,0.05,0.06,0.10,355,0.000000,0,0,0
2584,25.06,25.06,24.63,25.38,7,8,39,26,0.08,0.05,0.07,0.06,360,-0.015385,0,0,0
5608,25.19,25.19,25.00,25.88,14,16,64,46,0.13,0.05,0.14,0.10,355,0.000000,0,0,0
821,26.31,26.44,26.00,26.44,149,234,178,10,0.51,0.14,0.19,0.12,1055,2.819231,1,1,1
8695,25.50,25.44,25.13,25.56,0,0,0,0,0.07,0.04,0.07,0.08,380,-0.126923,0,0,0


In [48]:
test_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
9605,25.13,25.13,24.63,25.25,0,0,0,0,0.08,0.05,0.06,0.08,345,0.000000,0,0,0
2530,25.06,25.06,24.56,25.25,6,6,34,22,0.08,0.05,0.07,0.06,360,0.000000,0,0,0
4219,25.44,25.44,25.00,25.56,0,0,0,0,0.07,0.04,0.05,0.06,375,0.000000,0,0,0
8432,25.81,25.94,25.50,25.88,0,0,0,0,0.08,0.04,0.06,0.09,750,-2.092308,0,0,0
3402,26.13,25.88,25.56,26.31,119,24,195,17,0.79,0.78,2.42,0.48,570,2.126923,1,0,1


## Paso 5: División del *dataset* de entrenamiento en *features* y *output*

In [49]:
# Get features columns
X = preprocess.get_features(train_df)

# Get output column
y = preprocess.get_output(train_df)

In [50]:
X.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
5671,25.25,25.25,25.06,25.88,15,18,70,49,0.07,0.05,0.06,0.10,355,0.000000,0,0
2584,25.06,25.06,24.63,25.38,7,8,39,26,0.08,0.05,0.07,0.06,360,-0.015385,0,0
5608,25.19,25.19,25.00,25.88,14,16,64,46,0.13,0.05,0.14,0.10,355,0.000000,0,0
821,26.31,26.44,26.00,26.44,149,234,178,10,0.51,0.14,0.19,0.12,1055,2.819231,1,1
8695,25.50,25.44,25.13,25.56,0,0,0,0,0.07,0.04,0.07,0.08,380,-0.126923,0,0


In [51]:
y.head()

5671    0
2584    0
5608    0
821     1
8695    0
Name: Room_Occupancy_Count, dtype: int32

# 2. Entrenamiento y seguimiento de la huella de carbono con **CodeCarbon**

In [52]:
training.train_LR_codecarbon(X,y)
training.train_RF_codecarbon(X,y)
training.train_SVC_codecarbon(X,y)

[codecarbon INFO @ 10:45:16] [setup] RAM Tracking...
[codecarbon INFO @ 10:45:16] [setup] GPU Tracking...
[codecarbon INFO @ 10:45:16] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:45:16] [setup] CPU Tracking...
[codecarbon WARNING @ 10:45:16] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 10:45:18] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 10:45:18] >>> Tracker's metadata:
[codecarbon INFO @ 10:45:18]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 10:45:18]   Python version: 3.10.4
[codecarbon INFO @ 10:45:18]   Available RAM : 7.935 GB
[codecarbon INFO @ 10:45:18]   CPU count: 6
[codecarbon INFO @ 10:45:18]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 10:45:18]   GPU count: 1
[codecarbon INFO @ 10:45:18]   GPU model: 1 x NVIDIA GeForce GT 710
[codecarbon INFO @ 10:45:18] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W


Emissions: 9.13725593050998e-08 kg


[codecarbon INFO @ 10:45:20] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 10:45:21] >>> Tracker's metadata:
[codecarbon INFO @ 10:45:21]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 10:45:21]   Python version: 3.10.4
[codecarbon INFO @ 10:45:21]   Available RAM : 7.935 GB
[codecarbon INFO @ 10:45:21]   CPU count: 6
[codecarbon INFO @ 10:45:21]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 10:45:21]   GPU count: 1
[codecarbon INFO @ 10:45:21]   GPU model: 1 x NVIDIA GeForce GT 710
[codecarbon INFO @ 10:45:21] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W
[codecarbon INFO @ 10:45:21] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 0.0 W
[codecarbon INFO @ 10:45:21] Energy consumed for all CPUs : 0.000002 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 10:45:21] 0.000002 kWh of electricity used since the begining.
[codecarbon INFO @ 10:45:21] [setup] 

Emissions: 3.4333504698454613e-07 kg


[codecarbon INFO @ 10:45:23] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 10:45:23] >>> Tracker's metadata:
[codecarbon INFO @ 10:45:23]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 10:45:23]   Python version: 3.10.4
[codecarbon INFO @ 10:45:23]   Available RAM : 7.935 GB
[codecarbon INFO @ 10:45:23]   CPU count: 6
[codecarbon INFO @ 10:45:23]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 10:45:23]   GPU count: 1
[codecarbon INFO @ 10:45:23]   GPU model: 1 x NVIDIA GeForce GT 710
C:\Users\INTEL I5\miniconda3\envs\TFG_Project\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
[codecarbon INFO @ 10:45:23] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W
[codecarbon INFO @ 10:45:23] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 0.0 W
[codecarbon INFO @ 10:45

Emissions: 4.154400140245211e-08 kg
